# CREACIÓN DE LOS DATAFRAME PARA EL USO DE LAS FUNCIONES DE LA API

In [1]:
# IMPORTACIÓN DE LAS LIBRERIAS PARA EL PROCESO...

import pandas as pd
import os

In [2]:
# CARGA DE LOS DATAFRAMES DEL DATASET PARA LA CREACIÓN DE DATAFRAMES DE LAS FUNCIONES...

games_data = pd.read_csv("DATASET_STEAM_LIMPIO/games_limpio.csv")
items_data = pd.read_csv("DATASET_STEAM_LIMPIO/items_limpio.csv")
reviews_data = pd.read_csv("DATASET_STEAM_LIMPIO/reviews_limpio.csv")

## FUNCIÓN Nro. 01: def PlayTimeGenre...

In [3]:
# CREACIÓN DEL DATAFRAME "PlayTimeGenre_games" Y EL DATAFRAME "PlayTimeGenre_items"...

# Se crea un nuevo dataframe PlayTimeGenre_games a partir del dataframe games_data
PlayTimeGenre_games = games_data[["genres", "release_date", "id"]]
PlayTimeGenre_games = PlayTimeGenre_games.rename(columns={'id': 'item_id'})

# Se crea un nuevo dataframe PlayTimeGenre_items a partir del dataframe items_data
PlayTimeGenre_items = items_data[["item_id", "playtime_forever"]]
PlayTimeGenre_items['playtime_forever'] = PlayTimeGenre_items['playtime_forever'].apply(lambda x: 0.0 if pd.isna(x) or x == '' else float(x))

C:\Users\Yaide\AppData\Local\Temp\ipykernel_1896\3806362220.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PlayTimeGenre_items['playtime_forever'] = PlayTimeGenre_items['playtime_forever'].apply(lambda x: 0.0 if pd.isna(x) or x == '' else float(x))


In [4]:
# TRATAMIENTO DE LA COLUMNA "genres"...

# Transformación de los datos de la columna genres al tipo str
PlayTimeGenre_games['genres'] = PlayTimeGenre_games['genres'].apply(lambda x: 'SD' if pd.isna(x) or x == '' else str(x))

In [5]:
PlayTimeGenre_games

,genres,release_date,item_id
0,Action,2018-01-04,761140.0
1,Casual,2018-01-04,761140.0
2,Indie,2018-01-04,761140.0
3,Simulation,2018-01-04,761140.0
4,Strategy,2018-01-04,761140.0
...,...,...,...
74832,Racing,2018-01-04,610660.0
74833,Simulation,2018-01-04,610660.0
74834,Casual,2017-09-02,658870.0
74835,Indie,2017-09-02,658870.0


In [6]:
PlayTimeGenre_items

,item_id,playtime_forever
0,10.0,6.0
1,10.0,0.0
2,1200.0,923.0
3,10.0,0.0
4,300.0,1131.0
...,...,...
87627,413850.0,0.0
87628,11020.0,0.0
87629,NaN,0.0
87630,304930.0,677.0


In [7]:
# CREACIÓN DEL DATAFRAME "PlayTimeGenre_final" A PARTIR DE LA FUSIÓN DE LOS DOS DATAFRAMES...

# Fusiona los DataFrames en base a la columna "item_id"
PlayTimeGenre_final = PlayTimeGenre_items.merge(PlayTimeGenre_games, on="item_id", how="outer")

# Rellena los valores nulos con "SD"
PlayTimeGenre_final.fillna("SD", inplace=True)

In [8]:
PlayTimeGenre_final

,item_id,playtime_forever,genres,release_date
0,10.0,6.0,Action,2000-11-01
1,10.0,0.0,Action,2000-11-01
2,10.0,0.0,Action,2000-11-01
3,10.0,328.0,Action,2000-11-01
4,10.0,580.0,Action,2000-11-01
...,...,...,...,...
229366,610660.0,SD,Racing,2018-01-04
229367,610660.0,SD,Simulation,2018-01-04
229368,658870.0,SD,Casual,2017-09-02
229369,658870.0,SD,Indie,2017-09-02


In [9]:
# TRATAMIENTO DE LA COLUMNA "release_date" DEL DATAFRAME "PlayTimeGenre_final"...

# Convertir la columna "release_date" a formato datetime
PlayTimeGenre_final['release_date'] = pd.to_datetime(PlayTimeGenre_final['release_date'], errors='coerce')

# Extraer el año como un entero y reemplazar NaN con 0
PlayTimeGenre_final['release_date'] = PlayTimeGenre_final['release_date'].dt.year.fillna(0).astype(int)

In [10]:
# PROCESO DE TRATAMIENTO DEL DATAFRAME "PlayTimeGenre"...

# Supongamos que tienes un DataFrame llamado "PlayTimeGenre_final" y deseas eliminar las filas con "SD" en "release_date" y "playtime_forever"
PlayTimeGenre_final = PlayTimeGenre_final.drop(PlayTimeGenre_final[(PlayTimeGenre_final['release_date'] == 'SD') | (PlayTimeGenre_final['playtime_forever'] == 'SD')].index)

# Supongamos que tienes un DataFrame llamado "PlayTimeGenre" con la columna "playtime_forever"
PlayTimeGenre_final['playtime_forever'] = PlayTimeGenre_final['playtime_forever'].astype(int)

# Sumatoria de los valores de la columna "playtime_forever" por cada año y por cada género
PlayTimeGenre = PlayTimeGenre_final.groupby(['genres', 'release_date'])['playtime_forever'].sum().reset_index()

In [11]:
PlayTimeGenre

,genres,release_date,playtime_forever
0,Action,0,33642
1,Action,1990,2935
2,Action,1994,1649
3,Action,1995,2435
4,Action,1996,23
...,...,...,...
203,Strategy,2016,1263
204,Strategy,2017,1222
205,Utilities,2006,1725
206,Utilities,2014,1


## FUNCIÓN Nro. 02: def UserForGenre...

In [12]:
# CREACIÓN DEL DATAFRAME "UserForGenre"...

# Se crea un nuevo dataframe UserForGenre_games a partir del dataframe games_data
UserForGenre_games = games_data[["genres", "release_date", "id"]]
UserForGenre_games = UserForGenre_games.rename(columns={'id': 'item_id'})
UserForGenre_games['release_date'] = pd.to_datetime(UserForGenre_games['release_date'], errors='coerce')
UserForGenre_games['release_date'] = UserForGenre_games['release_date'].dt.year.fillna(0).astype(int)

# Se crea un nuevo dataframe UserForGenre_items a partir del dataframe items_data
UserForGenre_items = items_data[["user_id", "item_id", "playtime_forever"]]
UserForGenre_items['playtime_forever'] = UserForGenre_items['playtime_forever'].apply(lambda x: 0.0 if pd.isna(x) or x == '' else float(x))

C:\Users\Yaide\AppData\Local\Temp\ipykernel_1896\2184286571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UserForGenre_items['playtime_forever'] = UserForGenre_items['playtime_forever'].apply(lambda x: 0.0 if pd.isna(x) or x == '' else float(x))


In [13]:
UserForGenre_games

,genres,release_date,item_id
0,Action,2018,761140.0
1,Casual,2018,761140.0
2,Indie,2018,761140.0
3,Simulation,2018,761140.0
4,Strategy,2018,761140.0
...,...,...,...
74832,Racing,2018,610660.0
74833,Simulation,2018,610660.0
74834,Casual,2017,658870.0
74835,Indie,2017,658870.0


In [14]:
UserForGenre_items

,user_id,item_id,playtime_forever
0,76561197970982479,10.0,6.0
1,js41637,10.0,0.0
2,evcentric,1200.0,923.0
3,Riot-Punch,10.0,0.0
4,doctr,300.0,1131.0
...,...,...,...
87627,76561198323066619,413850.0,0.0
87628,76561198326700687,11020.0,0.0
87629,XxLaughingJackClown77xX,NaN,0.0
87630,76561198329548331,304930.0,677.0


In [15]:
# CREACIÓN DEL DATAFRAME "UserForGenre_final" A PARTIR DE LA FUSIÓN DE LOS DOS DATAFRAMES...

# Fusiona los DataFrames en base a la columna "item_id"
UserForGenre_final = UserForGenre_items.merge(UserForGenre_games, on="item_id", how="outer")

# Rellena los valores nulos con "SD"
UserForGenre_final.fillna("SD", inplace=True)

In [16]:
# PROCESO DE TRATAMIENTO DEL DATAFRAME "UserForGenre"...

# Eliminar del dataframe "UserForGenre_final" las filas con "SD" en "user_id", "release_date" y "playtime_forever"
columnas_a_verificar = ['user_id', 'playtime_forever', 'release_date', 'genres']
UserForGenre_final = UserForGenre_final[~UserForGenre_final[columnas_a_verificar].eq('SD').any(axis=1)]

# Formatear la columna 'release_date' para eliminar decimales
UserForGenre_final['release_date'] = UserForGenre_final['release_date'].astype('int64').astype('int32')

In [17]:
# Eliminación de la columna "item_id"
UserForGenre = UserForGenre_final.drop("item_id", axis=1)

In [18]:
UserForGenre

,user_id,playtime_forever,genres,release_date
0,76561197970982479,6.0,Action,2000
1,js41637,0.0,Action,2000
2,Riot-Punch,0.0,Action,2000
3,WeiEDKrSat,328.0,Action,2000
4,Fr0stedLine,580.0,Action,2000
...,...,...,...,...
156577,Oersted_the_Hopeful,0.0,RPG,2014
156578,Oersted_the_Hopeful,0.0,Strategy,2014
156579,JustMielThings,677.0,RPG,2015
156580,JustMielThings,677.0,Strategy,2015


## FUNCIÓN Nro. 03: def UsersRecommend...

In [19]:
# CREACIÓN DEL DATAFRAME "UsersRecommend"...

# Se crea un nuevo dataframe UsersRecommend_items a partir del dataframe items_data
UsersRecommend_items = items_data[["user_id", "item_id", "item_name"]]

# Se crea un nuevo dataframe UserForGenre_reviews a partir del dataframe reviews_data
UserForGenre_reviews = reviews_data[["user_id", "item_id", "review", "sentiment_analysis"]]

# Se crea un nuevo dataframe UserForGenre_gamess a partir del dataframe games_data
UserForGenre_games = games_data[["id", "release_date"]]
UserForGenre_games = UserForGenre_games.rename(columns={'id': 'item_id'})
UserForGenre_games['release_date'] = pd.to_datetime(UserForGenre_games['release_date'], errors='coerce')
UserForGenre_games['release_date'] = UserForGenre_games['release_date'].dt.year.fillna(0).astype(int)

In [20]:
UsersRecommend_items

,user_id,item_id,item_name
0,76561197970982479,10.0,Counter-Strike
1,js41637,10.0,Counter-Strike
2,evcentric,1200.0,Red Orchestra: Ostfront 41-45
3,Riot-Punch,10.0,Counter-Strike
4,doctr,300.0,Day of Defeat: Source
...,...,...,...
87627,76561198323066619,413850.0,CS:GO Player Profiles
87628,76561198326700687,11020.0,TrackMania Nations Forever
87629,XxLaughingJackClown77xX,NaN,NaN
87630,76561198329548331,304930.0,Unturned


In [21]:
UserForGenre_reviews

,user_id,item_id,review,sentiment_analysis
0,76561197970982479,1250,Simple yet with great replayability. In my opi...,2
1,js41637,251610,I know what you think when you see this title ...,2
2,evcentric,248820,A suitably punishing roguelike platformer. Wi...,2
3,doctr,250320,This game... is so fun. The fight sequences ha...,2
4,maplemage,211420,Git gud,1
...,...,...,...,...
25453,76561198306599751,261030,I cried in the end its so sadding ]'; I wish l...,2
25454,Ghoustik,730,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...,1
25455,76561198310819422,570,Well Done,1
25456,76561198312638244,233270,this is a very fun and nice 80s themed shooter...,2


In [22]:
UserForGenre_games

,item_id,release_date
0,761140.0,2018
1,761140.0,2018
2,761140.0,2018
3,761140.0,2018
4,761140.0,2018
...,...,...
74832,610660.0,2018
74833,610660.0,2018
74834,658870.0,2017
74835,658870.0,2017


In [23]:
# FUSIÓN Y TRATAMIENTO DE LOS TRES DATAFRAMES EN UNO SOLO...

# Fusiona los tres DataFrames en base a la columna "item_id"
UsersRecommend_final = UserForGenre_games.merge(UserForGenre_reviews, on="item_id", how="outer").merge(UsersRecommend_items, on="item_id", how="outer")

# Combina la columna "user_id" de "UsersRecommend_items" y "UserForGenre_reviews"
UsersRecommend_final["user_id"] = UsersRecommend_final["user_id_x"].combine_first(UsersRecommend_final["user_id_y"])
UsersRecommend_final.drop(["user_id_x", "user_id_y"], axis=1, inplace=True)

# Rellena los valores nulos con "SD"
UsersRecommend_final.fillna("SD", inplace=True)

# Limpieza del dataframe
UsersRecommend_final = UsersRecommend_final[(UsersRecommend_final != "SD").all(axis=1)]

# Convierte la columna "release_date" en valores enteros y rellena los valores no numéricos con 0
UsersRecommend_final["release_date"] = pd.to_numeric(UsersRecommend_final["release_date"], errors='coerce').fillna(0).astype(int)

# Convierte la columna "sentiment_analysis" en valores enteros y rellena los valores no numéricos con 0
UsersRecommend_final["sentiment_analysis"] = pd.to_numeric(UsersRecommend_final["sentiment_analysis"], errors='coerce').fillna(0).astype(int)

In [24]:
# ELIMINACIÓN DE DATOS REPETIDOS...

# para los valores repetidos
print(f'Tamaño del Dataset antes de eliminar filas repetidas: {UsersRecommend_final.shape}')
UsersRecommend_final.drop_duplicates(inplace=True)
print(f'Tamaño del Dataset después de eliminar filas repetidas: {UsersRecommend_final.shape}')

Tamaño del Dataset antes de eliminar filas repetidas: (35000997, 6)
Tamaño del Dataset después de eliminar filas repetidas: (15583, 6)


In [25]:
UsersRecommend_final

,item_id,release_date,review,sentiment_analysis,item_name,user_id
88,70.0,1998,ADVERTENCIA:Este Análisis contiene posibles Sp...,0,Half-Life,EizanAratoFujimaki
1248,70.0,1998,this game has had me hooked for over 4 years a...,0,Half-Life,76561198020928326
2408,70.0,1998,Juegazo que marco mi infancia y la de muchos a...,1,Half-Life,The-Dunktown-Express
3568,70.0,1998,some of the sound effects in this game can be ...,1,Half-Life,76561197994404698
4728,70.0,1998,I dont think you can run steam if you dont mee...,0,Half-Life,Engurneer
...,...,...,...,...,...,...
35142551,40.0,2001,Best playerbase I have ever met. Everybody is ...,2,Deathmatch Classic,2ez2xl
35142564,130.0,2001,"Half-Life 1'in ikinci resmi eklentisi, tıpkı ""...",1,Half-Life: Blue Shift,Scorpion1903
35142599,130.0,2001,Great and Awesome Game,2,Half-Life: Blue Shift,playergdgsd
35142744,13230.0,2004,good game to play its the old school games tha...,2,Unreal Tournament 2004,drkritisch


In [26]:
# Agrupar por 'item_name' y 'release_date', contar la cantidad de veces que aparece el valor "2" y el valor "0" en la columna 'sentiment_analysis'
UsersRecommend_groupby = UsersRecommend_final.groupby(['item_name', 'release_date'])['sentiment_analysis'].apply(lambda x: ((x == 2) | (x == 0)).sum()).reset_index(name='total_count_recommend')

In [27]:
# Ordenar el DataFrame por total_count_recommend en orden descendente
UsersRecommend = UsersRecommend_groupby.sort_values(by='total_count_recommend', ascending=False)

In [28]:
UsersRecommend

,item_name,release_date,total_count_recommend
121,Counter-Strike: Global Offensive,2012,1739
235,Garry's Mod,2006,565
379,PAYDAY 2,0,418
142,DayZ,2013,329
452,Rust,2013,327
...,...,...,...
110,Command and Conquer: Red Alert 3 - Uprising,2009,0
415,Quantum Conundrum,2012,0
211,Feeding Frenzy 2: Shipwreck Showdown Deluxe,2006,0
95,Champions Online,2009,0


## FUNCIÓN Nro. 04: def sentiment_analysis...

In [29]:
# CREACIÓN DEL DATAFRAME "sentimentanalysis"...

# Se crea un nuevo dataframe UsersRecommend_items a partir del dataframe items_data
sentimentanalysis_items = items_data[["user_id", "item_id", "item_name"]]

# Se crea un nuevo dataframe UserForGenre_reviews a partir del dataframe reviews_data
sentimentanalysis_reviews = reviews_data[["user_id", "item_id", "review", "sentiment_analysis"]]

# Se crea un nuevo dataframe UserForGenre_gamess a partir del dataframe games_data
sentimentanalysis_games = games_data[["id", "release_date"]]
sentimentanalysis_games = sentimentanalysis_games.rename(columns={'id': 'item_id'})
sentimentanalysis_games['release_date'] = pd.to_datetime(sentimentanalysis_games['release_date'], errors='coerce')
sentimentanalysis_games['release_date'] = sentimentanalysis_games['release_date'].dt.year.fillna(0).astype(int)

In [30]:
# FUSIÓN Y TRATAMIENTO DE LOS TRES DATAFRAMES EN UNO SOLO...

# Fusiona los tres DataFrames en base a la columna "item_id"
sentimentanalysis_final = sentimentanalysis_games.merge(sentimentanalysis_reviews, on="item_id", how="outer").merge(sentimentanalysis_items, on="item_id", how="outer")

# Combina la columna "user_id" de "sentimentanalysis_items" y "sentimentanalysis_reviews"
sentimentanalysis_final["user_id"] = sentimentanalysis_final["user_id_x"].combine_first(sentimentanalysis_final["user_id_y"])
sentimentanalysis_final.drop(["user_id_x", "user_id_y"], axis=1, inplace=True)

# Rellena los valores nulos con "SD"
sentimentanalysis_final.fillna("SD", inplace=True)

# Limpieza del dataframe
sentimentanalysis_final = sentimentanalysis_final[(sentimentanalysis_final != "SD").all(axis=1)]

# Convierte la columna "release_date" en valores enteros y rellena los valores no numéricos con 0
sentimentanalysis_final["release_date"] = pd.to_numeric(sentimentanalysis_final["release_date"], errors='coerce').fillna(0).astype(int)

# Convierte la columna "sentiment_analysis" en valores enteros y rellena los valores no numéricos con 0
sentimentanalysis_final["sentiment_analysis"] = pd.to_numeric(sentimentanalysis_final["sentiment_analysis"], errors='coerce').fillna(0).astype(int)

In [31]:
# ELIMINACIÓN DE DATOS REPETIDOS...

# para los valores repetidos
print(f'Tamaño del Dataset antes de eliminar filas repetidas: {sentimentanalysis_final.shape}')
sentimentanalysis_final.drop_duplicates(inplace=True)
print(f'Tamaño del Dataset después de eliminar filas repetidas: {sentimentanalysis_final.shape}')

Tamaño del Dataset antes de eliminar filas repetidas: (35000997, 6)
Tamaño del Dataset después de eliminar filas repetidas: (15583, 6)


In [32]:
sentimentanalysis_final

,item_id,release_date,review,sentiment_analysis,item_name,user_id
88,70.0,1998,ADVERTENCIA:Este Análisis contiene posibles Sp...,0,Half-Life,EizanAratoFujimaki
1248,70.0,1998,this game has had me hooked for over 4 years a...,0,Half-Life,76561198020928326
2408,70.0,1998,Juegazo que marco mi infancia y la de muchos a...,1,Half-Life,The-Dunktown-Express
3568,70.0,1998,some of the sound effects in this game can be ...,1,Half-Life,76561197994404698
4728,70.0,1998,I dont think you can run steam if you dont mee...,0,Half-Life,Engurneer
...,...,...,...,...,...,...
35142551,40.0,2001,Best playerbase I have ever met. Everybody is ...,2,Deathmatch Classic,2ez2xl
35142564,130.0,2001,"Half-Life 1'in ikinci resmi eklentisi, tıpkı ""...",1,Half-Life: Blue Shift,Scorpion1903
35142599,130.0,2001,Great and Awesome Game,2,Half-Life: Blue Shift,playergdgsd
35142744,13230.0,2004,good game to play its the old school games tha...,2,Unreal Tournament 2004,drkritisch


In [33]:
# Eliminar las columnas "item_id", "review" y "user_id"
columnas_a_eliminar = ['item_id', 'review', 'user_id']
SentimentAnalysis = sentimentanalysis_final.drop(columnas_a_eliminar, axis=1)

In [34]:
SentimentAnalysis

,release_date,sentiment_analysis,item_name
88,1998,0,Half-Life
1248,1998,0,Half-Life
2408,1998,1,Half-Life
3568,1998,1,Half-Life
4728,1998,0,Half-Life
...,...,...,...
35142551,2001,2,Deathmatch Classic
35142564,2001,1,Half-Life: Blue Shift
35142599,2001,2,Half-Life: Blue Shift
35142744,2004,2,Unreal Tournament 2004


## CARGA DE LOS ARCHIVOS EN UNA NUEVA CARPETA...

In [35]:
# Define la ruta de la carpeta
carpeta_destino = "Archivos_para_API"

# Verifica si la carpeta ya existe, si no, créala
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Guarda los DataFrames en archivos CSV dentro de la carpeta
PlayTimeGenre.to_csv(os.path.join(carpeta_destino, "PlayTimeGenre.csv"), index=False)
UserForGenre.to_csv(os.path.join(carpeta_destino, "UserForGenre.csv"), index=False)
UsersRecommend.to_csv(os.path.join(carpeta_destino, "UsersRecommend.csv"), index=False)
SentimentAnalysis.to_csv(os.path.join(carpeta_destino, "SentimentAnalysis.csv"), index=False)